We can explore the tokenizers from huggingface

In [1]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained("bert-base-cased")

# or

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/Users/michaelchen/Desktop/Programming/LLM_practice/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


we can try different tokenizers

In [3]:
print(tokenizer("Using a Transformer network is simple"))

print(tokenizer_bert("Using a Transformer network is simple"))

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}